In [252]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt 
import time
import random

config_file = 'configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
# url: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
#checkpoint_file = 'laboro_tomato_little_48ep.pth'
checkpoint_file = 'epoch_42.pth'

In [253]:
imagePath = 'lightedImages/angle1/2.png'
depthPath = 'lightedImages/angle1/2.npy'

#read the image
im = Image.open(imagePath)

# build the model from a config file and a checkpoint file
img = mmcv.imread(imagePath)
model = init_detector(config_file, checkpoint_file, device='cpu')
depthData = np.load(depthPath)
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

bbox_result, segm_result = result

labels = [
    np.full(bbox.shape[0], i, dtype=np.int32)\
    for i, bbox in enumerate(bbox_result)
]
labels = np.concatenate(labels)
bboxes = np.vstack(bbox_result)

segm_result = [x for x in segm_result if x != []]
segment = np.vstack(segm_result)

print(bboxes)

In [254]:
import cv2

x = cv2.imread(imagePath, cv2.IMREAD_COLOR)
count = 0
for bbox in bboxes:
    if bbox[4] > 0.67:
        count += 1
        x = cv2.rectangle(x, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
        cv2.putText(x, str(count), (int((int(bbox[0]) + int(bbox[2]))/2), int((int(bbox[1]) + int(bbox[3]))/2)), fontFace = cv2.FONT_HERSHEY_PLAIN, fontScale = 3, thickness = 3, color = (250,225,0))
        
cv2.imwrite('lightedImages/angle2/order.png', x)
plt.imshow(x)
print(count)

In [244]:
import pyrealsense2 as rs
color_intrin = rs.pyrealsense2.intrinsics() #create a color intrinsics property
color_intrin.width = 1280
color_intrin.height = 720
color_intrin.ppx = 646.9525146484375
color_intrin.ppy = 375.3297119140625
color_intrin.fx = 911.9494018554688
color_intrin.fy = 912.0560302734375
color_intrin.model = rs.pyrealsense2.distortion.inverse_brown_conrady
color_intrin.coeffs = [0.0, 0.0, 0.0, 0.0, 0.0]

In [246]:
from math import sqrt
 
# Function to find the circle on
# which the given three points lie
def findCircle(x1, y1, x2, y2, x3, y3) :
    x12 = x1 - x2;
    x13 = x1 - x3;
 
    y12 = y1 - y2;
    y13 = y1 - y3;
 
    y31 = y3 - y1;
    y21 = y2 - y1;
 
    x31 = x3 - x1;
    x21 = x2 - x1;
 
    # x1^2 - x3^2
    sx13 = pow(x1, 2) - pow(x3, 2);
 
    # y1^2 - y3^2
    sy13 = pow(y1, 2) - pow(y3, 2);
 
    sx21 = pow(x2, 2) - pow(x1, 2);
    sy21 = pow(y2, 2) - pow(y1, 2);
 
    f = (((sx13) * (x12) + (sy13) *
          (x12) + (sx21) * (x13) +
          (sy21) * (x13)) // (2 *
          ((y31) * (x12) - (y21) * (x13))));
             
    g = (((sx13) * (y12) + (sy13) * (y12) +
          (sx21) * (y13) + (sy21) * (y13)) //
          (2 * ((x31) * (y12) - (x21) * (y13))));
 
    c = (-pow(x1, 2) - pow(y1, 2) -
         2 * g * x1 - 2 * f * y1);
 
    # eqn of circle be x^2 + y^2 + 2*g*x + 2*f*y + c = 0
    # where centre is (h = -g, k = -f) and
    # radius r as r^2 = h^2 + k^2 - c
    h = -g;
    k = -f;
    sqr_of_r = h * h + k * k - c;
 
    # r is the radius
    r = round(sqrt(sqr_of_r), 5);
 
    # print("Centre = (", h, ", ", k, ")");
    # print("Radius = ", r);
    return r, [h,k]

In [250]:
def calculateSize(bboxes, segment, depthData):
    heightList = []
    widthList = []
    count = 0

    for bbox in bboxes:
        if bbox[4] > 0.7: #if detection accuracy is over 70%
            mask = np.asarray(segment[count], dtype="uint8") #binary matrix
            fil = [[-1,-1,-1],
                   [-1, 8,-1],
                   [-1,-1,-1]]

            output = convolve(mask,fil, mode='constant')
            edgeDepth =[]
            for y in range(len(output)):
                    for x in range(len(output[y])):
                            if output[y][x] > 0:
                                edgeDepth.append(depthData[y][x])
            centerDepth = median(edgeDepth)

            ### CIRCLE FITTING ###
            radiusList = []
            centerList = []

            for i in range(10000): #sample 3 random points and generate a circle
                randomList = random.sample(range(0, len(output[0])), 3)
                firstPoint = randomList[0]
                secondPoint = randomList[1]
                thirdPoint = randomList[2]
                try:
                    radius, center = findCircle(output[1][firstPoint], output[0][firstPoint], output[1][secondPoint], output[0][secondPoint], output[1][thirdPoint], output[0][thirdPoint])
                    radiusList.append(radius)
                    centerList.append(center)
                except ZeroDivisionError:
                    continue

            medianRadius = int(median(radiusList) + 0.5)

            xList = []
            yList = []

            for i in range(len(radiusList)):
                xList.append(centerList[i][0])
                yList.append(centerList[i][1])

            xMedian = int(median(xList) + 0.5) #center coordinates
            yMedian = int(median(yList) + 0.5) #center coordinates

            ### 3D distance measurement ###
            xMin = int(bbox[0])
            xMax = int(bbox[2])
            yMin = int(bbox[1])
            yMax = int(bbox[3])
            xMid = int((xMin + xMax)/2)
            yMid = int((yMin + yMax)/2)

            top = [xMid, yMin]
            bot = [xMid, yMax]
            left = [xMin, yMid]
            right = [xMax, yMid]
            topC = rs.rs2_deproject_pixel_to_point(color_intrin, top, centerDepth)
            botC = rs.rs2_deproject_pixel_to_point(color_intrin, bot, centerDepth)
            leftC = rs.rs2_deproject_pixel_to_point(color_intrin, left, centerDepth)
            rightC = rs.rs2_deproject_pixel_to_point(color_intrin, right, centerDepth)

            new_height = math.sqrt(math.pow(topC[0] - botC[0], 2) + math.pow(topC[1] - botC[1],2) + math.pow(topC[2] - botC[2], 2))*100
            new_width = math.sqrt(math.pow(leftC[0] - rightC[0], 2) + math.pow(leftC[1] - rightC[1],2) + math.pow(leftC[2] - rightC[2], 2))*100

            heightList.append(new_height*100)
            widthList.append(new_width*100)

            ### calculate how much the segmentation mask fills up the drawn circle ###
            maskPixelsInsideCircle = 0
            totalMaskPixels = 0
            nonZero = np.transpose(np.nonzero(mask))
            
            for length in range(len(nonZero)):
                totalMaskPixels += 1
                if (((yMedian - nonZero[length][0])**2 + (xMedian - nonZero[length][1])**2) > medianRadius**2):
                    continue
                else:
                    maskPixelsInsideCircle += 1
            circleArea = int(math.pi * medianRadius**2)
            fillPercentage = maskPixelsInsideCircle / circleArea
            fillPercentageList.append(fillPercentage)

        count = count + 1
    
    return heightList, widthList, fillPercentageList

In [ ]:
sizeList, fillPercentageList, errorList= calculateSize(bboxes, segment, depthData)
print(sizeList)
print(fillPercentageList)
print(errorList)

In [251]:
height = []
width = []
count = 0

for bbox in bboxes:
    if bbox[4] > 0.7:
        mask = np.asarray(segment[count], dtype="uint8") #binary matrix
        edgeDepth =[]
        for y in range(len(mask)):
                for x in range(len(mask[y])):
                        if mask[y][x] > 0:
                            edgeDepth.append(depthData[y][x])
        centerDepth = median(edgeDepth)

        xMin = int(bbox[0])
        xMax = int(bbox[2])
        yMin = int(bbox[1])
        yMax = int(bbox[3])
        xMid = int((xMin + xMax)/2)
        yMid = int((yMin + yMax)/2)

        top = [xMid, yMin]
        bot = [xMid, yMax]
        left = [xMin, yMid]
        right = [xMax, yMid]
        topC = rs.rs2_deproject_pixel_to_point(color_intrin, top, centerDepth)
        botC = rs.rs2_deproject_pixel_to_point(color_intrin, bot, centerDepth)
        leftC = rs.rs2_deproject_pixel_to_point(color_intrin, left, centerDepth)
        rightC = rs.rs2_deproject_pixel_to_point(color_intrin, right, centerDepth)

        new_height = math.sqrt(math.pow(topC[0] - botC[0], 2) + math.pow(topC[1] - botC[1],2) + math.pow(topC[2] - botC[2], 2))*100
        new_width = math.sqrt(math.pow(leftC[0] - rightC[0], 2) + math.pow(leftC[1] - rightC[1],2) + math.pow(leftC[2] - rightC[2], 2))*100

        height.append(new_height*100)
        width.append(new_width*100)
    
    count = count + 1
    
print(height)
print(width)

In [57]:
import math
from scipy.ndimage import convolve
from PIL import Image, ImageFilter
from numpy import median

# THIS FUNCTION CALCULATES THE TOMATO SIZE USING THE BOUNDING BOXES, SEGMENTATION AND DEPTHDATA #
# RETURNS A LIST OF SIZES AND CORRESPONDING LIST OF ERRORS - LOWER THE ERROR, THE MORE ACCURATE THE SIZE MEASUREMENT #

def calculateSize(bboxes, segment, depthData):
    sizeList = []
    errorList = []
    count = 0
    
    for bbox in bboxes:
        ### FIND EDGE DEPTH ###
        mask = np.asarray(segment[count], dtype="uint8") #get corresponding segmentation mask as binary matrix
        
        fil = [[-1,-1,-1],
               [-1, 8,-1],
               [-1,-1,-1]]
        
        output = convolve(mask, fil, mode='constant') #get edges
        
        edgeDepth = []

        for y in range(len(output)): 
                for x in range(len(output[y])):
                        if output[y][x] > 0:
                            edgeDepth.append(depthData[y][x])
        centerDepth = median(edgeDepth) #calculate median depth from tomato edges (this is our estimated center depth)
        print(len(edgeDepth))

        ### CIRCLE FITTING ###
        mask = Image.fromarray(mask * 255) #to convert to RGB
        filteredMask = mask.filter(ImageFilter.ModeFilter(size=10)) #apply mode filter to smoothen edges
        filteredMaskArray = np.asarray(filteredMask, dtype="uint8") 
        
        fil = [[-1,-1,-1],
               [-1, 8,-1],
               [-1,-1,-1]]
        
        output = convolve(filteredMaskArray, fil, mode='constant') #get edges
        
        edgePixels = []

        for y in range(len(output)): 
                for x in range(len(output[y])):
                        if output[y][x] > 0:
                            edgePixels.append([y,x])

        radiusList = []
        centerList = []

        for i in range(10000): #sample 3 random points and generate a circle
            randomList = random.sample(range(0, len(edgePixels)), 3)
            firstPoint = randomList[0]
            secondPoint = randomList[1]
            thirdPoint = randomList[2]
            try:
                radius, center = findCircle(edgePixels[firstPoint][1], edgePixels[firstPoint][0], edgePixels[secondPoint][1], edgePixels[secondPoint][0], edgePixels[thirdPoint][1], edgePixels[thirdPoint][0])
                radiusList.append(radius)
                centerList.append(center)
            except ZeroDivisionError:
                continue

        medianRadius = int(median(radiusList) + 0.5)

        xList = []
        yList = []

        for i in range(len(radiusList)):
            xList.append(centerList[i][0])
            yList.append(centerList[i][1])

        xMedian = int(median(xList) + 0.5) #center coordinates
        yMedian = int(median(yList) + 0.5) #center coordinates

        ### CALCULATE SIZE ###
        top = [xMedian, yMedian + medianRadius]
        bot = [xMedian, yMedian - medianRadius]
        topC = rs.rs2_deproject_pixel_to_point(color_intrin, top, centerDepth)
        botC = rs.rs2_deproject_pixel_to_point(color_intrin, bot, centerDepth)
        size = math.sqrt(math.pow(topC[0] - botC[0], 2) + math.pow(topC[1] - botC[1],2) + math.pow(topC[2] - botC[2], 2))*100
        sizeList.append(size)
        count = count + 1

        ### Calculate how many mask pixels are outside the drawn circle ###
        maskPixelsOutsideCircle = 0
        totalMaskPixels = 0

        for y in range(len(filteredMaskArray)):
            for x in range(len(filteredMaskArray[y])):
                if filteredMaskArray[y][x] > 0:
                    totalMaskPixels += 1
                    if (((yMedian - y)**2 + (xMedian - x)**2) > medianRadius**2):
                        maskPixelsOutsideCircle += 1
        error = maskPixelsOutsideCircle / totalMaskPixels
        errorList.append(error)
        
    return sizeList, errorList

In [58]:
sizeList, errorList= calculateSize(bboxes, segment, depthData)
print(sizeList)
print(errorList)

In [ ]:
#roundness estimation

height = []
width = []
start = time.clock()
for bbox in bboxes:
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    
    top = [xMid, yMin]
    bot = [xMid, yMax]
    left = [xMin, yMid]
    right = [xMax, yMid]
    center = [xMid, yMid]
    
    pixelWidth = int((xMax - xMin))
    pixelHeight = int((yMax - yMin))
    centerDepth = depthData[yMid, xMid] * 1000

    f = 1.93

    Di = pixelWidth*2.453/1280
    Li = pixelHeight*2.453/1280

    calcWidth = centerDepth*Di/(f-Di/2)
    calcHeight = centerDepth*Li/(f-Li/2)
    height.append(calcWidth)
    width.append(calcHeight)
    
end = time.clock()
print(height)
print(width)
print(end - start)

In [101]:
tomatoSegment = segment[4]
plt.imshow(tomatoSegment)

In [102]:
tomatoSegment = segment[3]
plt.imshow(tomatoSegment)

In [103]:
tomatoSegment = segment[2]
plt.imshow(tomatoSegment)

In [110]:
from scipy.ndimage import convolve
from PIL import Image, ImageFilter
mask = np.asarray(tomatoSegment, dtype="uint8") #binary matrix
cv.imwrite('idealImages/segment.png', 255*mask)

In [111]:
mask = Image.open('idealImages/segment.png')
mask = mask.filter(ImageFilter.ModeFilter(size=20))
mask.save('idealImages/segment.png')

In [112]:
fil = [[-1,-1,-1],
       [-1, 8,-1],
       [-1,-1,-1]]

output = convolve(mask, fil, mode='constant')
plt.imshow(output)

In [113]:
image = plt.imread('idealImages/segment.png')
plt.imshow(image)

In [114]:
edgeDepth =[]
for y in range(len(output)):
        for x in range(len(output[y])):
                if output[y][x] > 0:
                    edgeDepth.append([y,x])
centerDepth = median(edgeDepth)
print(centerDepth)

In [115]:
print(len(edgeDepth))

In [116]:
radiusList = []
centerList = []
from numpy import median, mean
start = time.clock()
for i in range(10000):
    randomList = random.sample(range(0, len(edgeDepth)), 3)
    firstPoint = randomList[0]
    secondPoint = randomList[1]
    thirdPoint = randomList[2]
    try:
        radius, center = findCircle(edgeDepth[firstPoint][1], edgeDepth[firstPoint][0], edgeDepth[secondPoint][1], edgeDepth[secondPoint][0], edgeDepth[thirdPoint][1], edgeDepth[thirdPoint][0])
        radiusList.append(radius)
        centerList.append(center)
    except ZeroDivisionError:
        continue
end = time.clock()
print(end - start)
medianRadius = median(radiusList)
print(medianRadius)
medianRadius = int(medianRadius + 0.5)

xList = []
yList = []

for i in range(len(radiusList)):
    xList.append(centerList[i][0])
    yList.append(centerList[i][1])
    
xMedian = int(median(xList))
yMedian = int(median(yList))
print(xMedian)
print(yMedian)
#index = radiusList.index(median)  
#print(centerList[index])

In [117]:
from PIL import Image

color = (255, 0, 0)
image = cv.imread('idealImages/segment.png')
image = cv.circle(image, (xMedian, yMedian), int(medianRadius), color, 2)
#image = cv.circle(image, (centerList[index][0], centerList[index][1]), int(median), color, 2)
cv.imwrite('idealImages/segmentBorder.png', image)
plt.imshow(image)

In [208]:
print(bboxes)

In [10]:
import pyrealsense2 as rs
color_intrin = rs.pyrealsense2.intrinsics() #create a color intrinsics property
color_intrin.width = 1280
color_intrin.height = 720
color_intrin.ppx = 646.9525146484375
color_intrin.ppy = 375.3297119140625
color_intrin.fx = 911.9494018554688
color_intrin.fy = 912.0560302734375
color_intrin.model = rs.pyrealsense2.distortion.inverse_brown_conrady
color_intrin.coeffs = [0.0, 0.0, 0.0, 0.0, 0.0]

In [11]:
f = 1.93
pixelWidth = 119
centerDepth = 400
Di = pixelWidth*2.453/1280
calcWidth = centerDepth*Di/(f-Di/2)
print(calcWidth)

In [12]:
height = []
width = []
start = time.clock()
for bbox in bboxes:
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    
    top = [xMid, yMin]
    bot = [xMid, yMax]
    left = [xMin, yMid]
    right = [xMax, yMid]
    center = [xMid, yMid]
    
    pixelWidth = int((xMax - xMin))
    pixelHeight = int((yMax - yMin))
    centerDepth = depthData[yMid, xMid] * 1000

    f = 1.93

    Di = pixelWidth*2.453/1280
    Li = pixelHeight*2.453/1280

    calcWidth = centerDepth*Di/(f-Di/2)
    calcHeight = centerDepth*Li/(f-Li/2)
    height.append(calcWidth)
    width.append(calcHeight)
    
end = time.clock()
print(height)
print(width)
print(end - start)

In [6]:
from scipy.ndimage import convolve
from numpy import median

height = []
width = []
count = 0
start = time.clock()
for bbox in bboxes:
    mask = np.asarray(segment[count], dtype="uint8") #binary matrix
    fil = [[-1,-1,-1],
           [-1, 8,-1],
           [-1,-1,-1]]

    output = convolve(mask,fil, mode='constant')
    edgeDepth =[]
    for y in range(len(output)):
            for x in range(len(output[y])):
                    if output[y][x] > 0:
                        edgeDepth.append(depthData[y][x])
    centerDepth = median(edgeDepth) * 1000
    
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    
    top = [xMid, yMin]
    bot = [xMid, yMax]
    left = [xMin, yMid]
    right = [xMax, yMid]
    center = [xMid, yMid]
    
    pixelWidth = int((xMax - xMin))
    pixelHeight = int((yMax - yMin))

    f = 1.93

    Di = 1.05*pixelWidth*2.453/1280
    Li = 1.09*pixelHeight*2.453/1280
    
    calcWidth = centerDepth*Di/f
    calcHeight = centerDepth*Li/f
    height.append(calcWidth)
    width.append(calcHeight)
    count = count + 1
    
end = time.clock()
print(height)
print(width)
print(end - start)

In [5]:
# Calculating the fruit radius
import math
height = []
width = []
perPixelFOV_Radians_X = 1.50098/1280
perPixelFOV_Radians_Y = 0.994838/720

start = time.clock()

for bbox in bboxes:
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    
    top = [xMid, yMin]
    bot = [xMid, yMax]
    left = [xMin, yMid]
    right = [xMax, yMid]
    center = [xMid, yMid]
    
    fruitRadiusinPixelsX = int((xMax - xMin)/2)
    fruitRadiusinPixelsY = int((yMax - yMin)/2)
    centerDepth = depthData[yMid, xMid]

    FRUIT_RADIUSx = 200*(centerDepth)*math.tan(perPixelFOV_Radians_X*fruitRadiusinPixelsX)
    FRUIT_RADIUSy = 200*(centerDepth)*math.tan(perPixelFOV_Radians_Y*fruitRadiusinPixelsY)
    
    height.append(FRUIT_RADIUSy)
    width.append(FRUIT_RADIUSx)
end = time.clock()
print(height)
print(width)
print(end - start)

In [16]:
from PIL import Image
height = []
width = []
count = 0

start = time.clock()

for bbox in bboxes:
    mask = np.asarray(segment[count], dtype="uint8") #binary matrix

    # Find contours
    cnts = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c = max(cnts, key=cv.contourArea)

    # Obtain outer coordinates
    left = tuple(c[c[:, :, 0].argmin()][0])
    right = tuple(c[c[:, :, 0].argmax()][0])
    top = tuple(c[c[:, :, 1].argmin()][0])
    bottom = tuple(c[c[:, :, 1].argmax()][0])
    
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    center = [xMid, yMid]
    
    fruitRadiusinPixelsX = int((xMax - xMin)/2)
    fruitRadiusinPixelsY = int((yMax - yMin)/2)
    centerDepth = depthData[yMid, xMid]
    
    result = rs.rs2_deproject_pixel_to_point(color_intrin, center, centerDepth)
    distance = np.linalg.norm(result)

    FRUIT_RADIUSx = 200*(centerDepth)*math.tan(perPixelFOV_Radians_X*fruitRadiusinPixelsX)
    FRUIT_RADIUSy = 200*(centerDepth)*math.tan(perPixelFOV_Radians_Y*fruitRadiusinPixelsY)
    
    height.append(FRUIT_RADIUSy)
    width.append(FRUIT_RADIUSx)
    
    count = count + 1
end = time.clock()
print(end - start)
print(height)
print(width)

In [67]:
# build the model from a config file and a checkpoint file
img = mmcv.imread('image/83a1d79d46cf4f8a923bfbedc34f8a92 color.png')
model = init_detector(config_file, checkpoint_file, device='cpu')
depthData = np.load('image/83a1d79d46cf4f8a923bfbedc34f8a92 depthData.npy')
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

bbox_result, segm_result = result

labels = [
    np.full(bbox.shape[0], i, dtype=np.int32)\
    for i, bbox in enumerate(bbox_result)
]
labels = np.concatenate(labels)
bboxes = np.vstack(bbox_result)
segm_result = [x for x in segm_result if x != []]
segment = np.vstack(segm_result)

In [68]:
# Calculating the fruit radius
import math
height = []
width = []
perPixelFOV_Radians_X = 1.50098/1280
perPixelFOV_Radians_Y = 0.994838/720

for bbox in bboxes:
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    
    top = [xMid, yMin]
    bot = [xMid, yMax]
    left = [xMin, yMid]
    right = [xMax, yMid]
    center = [xMid, yMid]
    
    fruitRadiusinPixelsX = xMid - xMin
    fruitRadiusinPixelsY = yMid - yMin
    centerDepth = depthData[yMid, xMid]
    
    result = rs.rs2_deproject_pixel_to_point(color_intrin, center, centerDepth)
    #distance = np.linalg.norm(result)

    FRUIT_RADIUSx = 200*(centerDepth)*math.tan(perPixelFOV_Radians_X*fruitRadiusinPixelsX)
    FRUIT_RADIUSy = 200*(centerDepth)*math.tan(perPixelFOV_Radians_Y*fruitRadiusinPixelsY)
    
    height.append(FRUIT_RADIUSy)
    width.append(FRUIT_RADIUSx)
print(height)
print(width)

In [69]:
height = []
width = []
count = 0
for bbox in bboxes:
    mask = np.asarray(segment[count], dtype="uint8") #binary matrix

    # Find contours
    cnts = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c = max(cnts, key=cv.contourArea)

    # Obtain outer coordinates
    left = tuple(c[c[:, :, 0].argmin()][0])
    right = tuple(c[c[:, :, 0].argmax()][0])
    top = tuple(c[c[:, :, 1].argmin()][0])
    bottom = tuple(c[c[:, :, 1].argmax()][0])
    
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    center = [xMid, yMid]
    
    fruitRadiusinPixelsX = xMid - left[0]
    fruitRadiusinPixelsY = yMid - top[1]
    centerDepth = depthData[yMid, xMid]
    
    result = rs.rs2_deproject_pixel_to_point(color_intrin, center, centerDepth)
    #distance = np.linalg.norm(result)

    FRUIT_RADIUSx = 200*(centerDepth)*math.tan(perPixelFOV_Radians_X*fruitRadiusinPixelsX)
    FRUIT_RADIUSy = 200*(centerDepth)*math.tan(perPixelFOV_Radians_Y*fruitRadiusinPixelsY)
    
    height.append(FRUIT_RADIUSy)
    width.append(FRUIT_RADIUSx)
    
    count = count + 1

print(height)
print(width)

In [70]:
# build the model from a config file and a checkpoint file
img = mmcv.imread('image/374a7430eca74a89b741a537b8f30e15 color.png')
model = init_detector(config_file, checkpoint_file, device='cpu')
depthData = np.load('image/374a7430eca74a89b741a537b8f30e15 depthData.npy')
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

bbox_result, segm_result = result

labels = [
    np.full(bbox.shape[0], i, dtype=np.int32)\
    for i, bbox in enumerate(bbox_result)
]
labels = np.concatenate(labels)
bboxes = np.vstack(bbox_result)
segm_result = [x for x in segm_result if x != []]
segment = np.vstack(segm_result)

In [71]:
# Calculating the fruit radius
import math
height = []
width = []
perPixelFOV_Radians_X = 1.50098/1280
perPixelFOV_Radians_Y = 0.994838/720

for bbox in bboxes:
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    
    top = [xMid, yMin]
    bot = [xMid, yMax]
    left = [xMin, yMid]
    right = [xMax, yMid]
    center = [xMid, yMid]
    
    fruitRadiusinPixelsX = xMid - xMin
    fruitRadiusinPixelsY = yMid - yMin
    centerDepth = depthData[yMid, xMid]
    
    result = rs.rs2_deproject_pixel_to_point(color_intrin, center, centerDepth)
    #distance = np.linalg.norm(result)

    FRUIT_RADIUSx = 200*(centerDepth)*math.tan(perPixelFOV_Radians_X*fruitRadiusinPixelsX)
    FRUIT_RADIUSy = 200*(centerDepth)*math.tan(perPixelFOV_Radians_Y*fruitRadiusinPixelsY)
    
    height.append(FRUIT_RADIUSy)
    width.append(FRUIT_RADIUSx)
print(height)
print(width)

In [72]:
height = []
width = []
count = 0
for bbox in bboxes:
    mask = np.asarray(segment[count], dtype="uint8") #binary matrix

    # Find contours
    cnts = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c = max(cnts, key=cv.contourArea)

    # Obtain outer coordinates
    left = tuple(c[c[:, :, 0].argmin()][0])
    right = tuple(c[c[:, :, 0].argmax()][0])
    top = tuple(c[c[:, :, 1].argmin()][0])
    bottom = tuple(c[c[:, :, 1].argmax()][0])
    
    xMin = int(bbox[0])
    xMax = int(bbox[2])
    yMin = int(bbox[1])
    yMax = int(bbox[3])
    xMid = int((xMin + xMax)/2)
    yMid = int((yMin + yMax)/2)
    center = [xMid, yMid]
    
    fruitRadiusinPixelsX = xMid - left[0]
    fruitRadiusinPixelsY = yMid - top[1]
    centerDepth = depthData[yMid, xMid]
    
    result = rs.rs2_deproject_pixel_to_point(color_intrin, center, centerDepth)
    #distance = np.linalg.norm(result)

    FRUIT_RADIUSx = 200*(centerDepth)*math.tan(perPixelFOV_Radians_X*fruitRadiusinPixelsX)
    FRUIT_RADIUSy = 200*(centerDepth)*math.tan(perPixelFOV_Radians_Y*fruitRadiusinPixelsY)
    
    height.append(FRUIT_RADIUSy)
    width.append(FRUIT_RADIUSx)
    
    count = count + 1

print(height)
print(width)